In [2]:
# importing libraries
import numpy as np
import time
import math
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = 200

import warnings
warnings.filterwarnings("ignore")
from catboost import CatBoostClassifier
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [4]:
raw_data = pd.read_csv('train.csv')
print(f"Dataset shape: {raw_data.shape}")
raw_data.head()

Dataset shape: (17996, 17)


,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


In [6]:
raw_data['key'].value_counts()

7.0     2097
2.0     1994
9.0     1961
1.0     1680
4.0     1549
11.0    1476
5.0     1397
6.0     1190
8.0     1119
10.0    1010
3.0      509
Name: key, dtype: int64

In [7]:
raw_data['Class'].value_counts()

10    4949
6     2587
9     2524
8     1854
5     1447
1     1373
2     1272
0      625
7      576
3      402
4      387
Name: Class, dtype: int64

In [8]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Artist Name         17996 non-null  object 
 1   Track Name          17996 non-null  object 
 2   Popularity          17568 non-null  float64
 3   danceability        17996 non-null  float64
 4   energy              17996 non-null  float64
 5   key                 15982 non-null  float64
 6   loudness            17996 non-null  float64
 7   mode                17996 non-null  int64  
 8   speechiness         17996 non-null  float64
 9   acousticness        17996 non-null  float64
 10  instrumentalness    13619 non-null  float64
 11  liveness            17996 non-null  float64
 12  valence             17996 non-null  float64
 13  tempo               17996 non-null  float64
 14  duration_in min/ms  17996 non-null  float64
 15  time_signature      17996 non-null  int64  
 16  Clas

In [9]:
raw_data.isnull().sum()

Artist Name              0
Track Name               0
Popularity             428
danceability             0
energy                   0
key                   2014
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness      4377
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
Class                    0
dtype: int64

In [10]:
#imput missing with zero
data = raw_data.fillna(0)

In [30]:
data.isnull().sum()

Artist Name           0
Track Name            0
Popularity            0
danceability          0
energy                0
key                   0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
duration_in min/ms    0
time_signature        0
Class                 0
dtype: int64

In [55]:
data['Track Name'].nunique()

15129

In [61]:
cat_col = ['Artist Name','Track Name']

for i in cat_col:
    data[i] = data[i].astype('str') 

In [62]:
x_train = data.drop('Class' ,axis = 1)
y_train = data['Class']

# submission 2

In [69]:
test_data = pd.read_csv('test.csv')
test_data = test_data.fillna(0)

In [74]:
# ideal parameters
clf = CatBoostClassifier(loss_function='MultiClass',
                         #eval_metric = 'Logloss',
                         #learning_rate = 0.9,
                         #l2_leaf_reg = 17,
                         #rsm = 0.8,
                         #scale_pos_weight = estimate,
                         #custom_loss = 'F1',
                         #iterations = 100,
                         verbose=False
                        )
clf.fit(x_train,y_train , cat_features = cat_col)

In [76]:
y_pred2 = clf.predict_proba(test_data)

In [77]:
y_pred2[0]

array([6.69845542e-05, 2.11424139e-01, 6.35946207e-03, 1.33700685e-05,
       2.02112236e-05, 4.81802338e-04, 2.42754112e-01, 1.46369373e-05,
       1.61416865e-02, 1.62511262e-02, 5.06472470e-01])

In [78]:
sub_temp = pd.read_csv("submission.csv")
finalpreds = pd.DataFrame(y_pred2)
finalpreds.columns = sub_temp.columns
finalpreds.to_csv("submit.csv",index=False)